# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [50]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [51]:
weather_df = pd.read_csv('weather_output/weather.csv')

weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.1203,-134.9692,76.73,78,30,10.29,PF,1645381136
1,1,Chui,-33.6971,-53.4616,82.02,58,77,14.67,UY,1645381197
2,2,Hermanus,-34.4187,19.2345,65.80,85,0,9.01,ZA,1645381091
3,3,Saldanha,-33.0117,17.9442,63.86,88,9,9.51,ZA,1645381198
4,4,Traiguén,-38.2500,-72.6833,73.09,30,23,7.70,CL,1645381198
...,...,...,...,...,...,...,...,...,...,...
584,638,Radovitskiy,55.1282,39.7956,27.36,92,53,8.72,RU,1645383207
585,639,Boden,65.8252,21.6886,14.00,98,100,3.76,SE,1645383207
586,640,Maryville,35.7565,-83.9705,60.15,31,5,5.99,US,1645383207
587,641,Buin,-33.7333,-70.7500,84.06,19,100,10.25,CL,1645383208


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [52]:
gmaps.configure(api_key=g_key)

In [74]:
#RUN THESE IN PYTHON DATA
#jupyter nbextension enable --py gmaps

locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = )

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [81]:
temp_pref = weather_df["Max Temp"] < 85
humid_pref = weather_df['Humidity'] < 50
cloud_pref = weather_df['Cloudiness'] > 30

hotel_df = weather_df.loc[temp_pref & humid_pref & cloud_pref]
df_len = len(hotel_df['City'])

hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,Ubari,26.5921,12.7805,65.98,15,72,9.91,LY,1645381198
11,12,Katsuura,35.1333,140.3000,39.22,48,77,17.90,JP,1645381200
45,49,Raymond,43.9015,-70.4703,26.42,36,36,7.45,US,1645381212
70,79,Huilong,31.8111,121.6550,38.03,49,100,3.65,CN,1645381280
93,104,Najrān,17.4924,44.1277,75.85,28,93,5.28,SA,1645381347


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
test_input = 'hotel'

test_lat = hotel_df.iloc[0,2]
test_lng = hotel_df.iloc[0,3]
test_url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={test_input}&inputtype=textquery&fields=name,geometry,address_component&locationbias=circle:5000@{test_lat},{test_lng}&key={g_key}'
    
test_response = requests.get(test_url).json()


In [56]:
pprint(test_response)

{'candidates': [],
 'error_message': "Error while parsing 'fields' parameter: Unsupported field "
                  "name 'address_component'. ",
 'status': 'INVALID_REQUEST'}


In [57]:
input = 'hotel'

hotel_names = []
hotel_lat = []
hotel_lng = []

for x in range(df_len):
    lat = hotel_df.iloc[x,2]
    lng = hotel_df.iloc[x,3]
    url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={input}&inputtype=textquery&language=en&fields=name,geometry&locationbias=circle:5000@{lat},{lng}&key={g_key}'
    
    response = requests.get(url).json()
    hotel_lat.append(response['candidates'][0]['geometry']['location']['lat'])
    hotel_lng.append(response['candidates'][0]['geometry']['location']['lng'])
    hotel_names.append(response['candidates'][0]['name'])



In [82]:
hotel_df['Hotel Name'] = hotel_names
hotel_df['Hotel Lat'] = hotel_lat
hotel_df['Hotel Lng'] = hotel_lng

hotel_df.head()

C:\Users\irish\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\irish\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\irish\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
6,6,Ubari,26.5921,12.7805,65.98,15,72,9.91,LY,1645381198,Hotels,29.500411,-98.717121
11,12,Katsuura,35.1333,140.3000,39.22,48,77,17.90,JP,1645381200,Sayan Terrace Hotel,35.181838,140.349655
45,49,Raymond,43.9015,-70.4703,26.42,36,36,7.45,US,1645381212,Microtel Inn & Suites by Wyndham Windham,43.857226,-70.452078
70,79,Huilong,31.8111,121.6550,38.03,49,100,3.65,CN,1645381280,Qidong Perfect International Hotel,31.822733,121.626134
93,104,Najrān,17.4924,44.1277,75.85,28,93,5.28,SA,1645381347,Clouds and stars of residential units,17.520916,44.194266


In [76]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

#hotel_info

In [77]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
                      
fig                

# Display figure



Figure(layout=FigureLayout(height='420px'))